In [1]:
import pandas as pd
from os import path
from os import listdir
import time
import numpy as np
from itertools import combinations

class FrequencyAnalys:

    def __init__(self,datapath,cat_path,columns_dict):

        files=[file for file in listdir(datapath) if '.csv' in file]
        self.datafiles=[path.join(datapath,filename) for filename in files ]
        category_df=pd.read_excel(cat_path)
        self.category_dict= dict(zip(category_df.cat, category_df.id_cat))
        self.columns_dict=columns_dict
    def preprocess_df(self,df_path):
        data=pd.read_csv(df_path,sep=';',encoding='utf-8')
        data.columns= [ self.columns_dict[el] for el in data.columns.values]
        unique_checks=data['check_id'].unique()
        checks_dict= dict(zip(unique_checks, list(range(len(unique_checks)))))
        data['h2']=data['h2'].apply(lambda x:self.category_dict[x])
        data['check_id']=data['check_id'].apply(lambda x:checks_dict[x])
        #data.drop('sales',1,inplace=True)
        data['sales']=1
        print(data.info())
        return data,len(unique_checks)
    
    def __frequency_filter(self,frequecy_series,window):
        #sorted_freq=fr.check_id
        sorted_freq=frequecy_series
        max_freq=sorted_freq[0]
        b=len(sorted_freq)-1
        a=1
        n=window
        x=int((a + b) / 2)
        f=lambda i:sum(sorted_freq[i:i+n])
        n_iter=n
        while (max_freq-f(x))*(max_freq-f(x-1))>0 and n_iter >0 :
            x=int((a + b) / 2)
            a, b = (a, x) if max_freq-f(x) > 0 else  (x, b)
            n_iter=n_iter-1
        return x
    
    
    
    def TURF(self,n,description=False):
        finalres=[]
        for datafile in self.datafiles:
            res=[]
            df,unique_checks_count=self.preprocess_df(datafile)
            freq_pt=pd.DataFrame(df.pivot_table(values='check_id',index=['h2'], 
                              aggfunc=lambda x: len(x.unique())))

            freq_pt.sort_values(by=['check_id'], ascending=[False],inplace=True)
            freq_pt.reset_index(inplace=True)
            x=self.__frequency_filter(freq_pt.check_id,n)
            cats=freq_pt.h2[:x]
            combo=combinations(list(cats),n)
            i=0
            start=time.time()
            for el in combo:
                i=i+1
                uniq_count=len(df[df['h2'].isin(el) ].check_id.unique())
                res.append([el,uniq_count])
            end=time.time()
            res=pd.DataFrame(res,columns=['combo','frequency']).sort_values(by =['frequency'], ascending=False)
            res.reset_index(inplace=True)
            res.drop('index',1,inplace=True)
            res['file']=path.splitext(path.basename(datafile))[0]
            res['checks_count']=unique_checks_count
            print('Processed {0} combinations in {1}'.format(i,end-start) )
            finalres.append(res.ix[:n*2])
        final=pd.concat(finalres)
        if description:
            inverse_dict=dict(zip(self.category_dict.values(), self.category_dict.keys()))
            final['description']=final['combo'].apply(lambda x:'-'.join([inverse_dict[el] for el in x]))
        return final
    
    def __convert_set_to_int(self,c_set,mask):
        res=0
        l=list(c_set)
        l.sort()
        for i in range(len(l)):
            res=res+l[i]*mask**i
        return res

    def __convert_int_to_set(self,c_int,mask):
        res=[]
        while c_int >0:
            res.append(c_int%mask)
            c_int=c_int//mask
        return set(res)
   
    
    
    
    def noTURF(self,n,mask,description=False):
        finalres=[]
        for datafile in self.datafiles:
            res=[]
            df,unique_checks_count=self.preprocess_df(datafile)
            
            pt=pd.DataFrame(df.pivot_table(values='h2',index=['check_id'], aggfunc=lambda x: set(x)))
            pt.reset_index(inplace=True)
            rename_check={'check_id':'check_id','h2':'h2_set'}
            pt.columns= [rename_check[el] for el in pt.columns.values]
            
            pt['power']=pt['h2_set'].apply(lambda x:len(x))
            pt=pt[pt['power']<=n]
    
            sets_n=pt[pt['power']==n].h2_set.values
            sets_to_integers=set([self.__convert_set_to_int(el,mask) for el in sets_n])
            
            uniq_sets=[self.__convert_int_to_set(el,mask) for el in sets_to_integers]
            sourse_sets=np.array(pt.h2_set.values)
            print('combos total ',len(uniq_sets))
            start=time.time()
            for el in uniq_sets:
                c=0
                for k in sourse_sets:
                    if k<= el:c=c+1

                res.append([el,c])
            end=time.time()
            res=pd.DataFrame(res,columns=['combo','frequency']).sort_values(by =['frequency'], ascending=False)
            res.reset_index(inplace=True)
            res.drop('index',1,inplace=True)
            res['file']=path.splitext(path.basename(datafile))[0]
            res['checks_count']=unique_checks_count
            print('Processed {0} combinations in {1}'.format(len(uniq_sets),end-start) )
            
            finalres.append(res.ix[:n*2])
             
        final=pd.concat(finalres)
        if description:
            inverse_dict=dict(zip(self.category_dict.values(), self.category_dict.keys()))
            final['description']=final['combo'].apply(lambda x:'-'.join([inverse_dict[el] for el in x]))
        return final
    
        

In [2]:
%%time
folder=r'C:\Reports\TURF'
datafolder=r'C:\Reports\TURF\Raw'
rename_dict={'Уникальный идентификатор':'check_id',
             'Продажи с НДС (SUM)':'sales',
             'Наименование Товарная категория 2':'h2'
            }

#fq=fa.FrequencyAnalys(datafolder,path.join(folder,'cat.xlsx'),rename_dict)
fq=FrequencyAnalys(datafolder,path.join(folder,'cat.xlsx'),rename_dict)
#fr=fq.TURF(3,description=True)
#fr=fq.noTURF(2,100,description=True)

Wall time: 386 ms


In [3]:
fr=fq.TURF(3,description=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12173437 entries, 0 to 12173436
Data columns (total 3 columns):
check_id    int64
sales       int64
h2          int64
dtypes: int64(3)
memory usage: 278.6 MB
None
Processed 35 combinations in 12.085999965667725
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3817573 entries, 0 to 3817572
Data columns (total 3 columns):
check_id    int64
sales       int64
h2          int64
dtypes: int64(3)
memory usage: 87.4 MB
None
Processed 20 combinations in 1.9590001106262207
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54610 entries, 0 to 54609
Data columns (total 3 columns):
check_id    54610 non-null int64
h2          54610 non-null int64
sales       54610 non-null int64
dtypes: int64(3)
memory usage: 1.2 MB
None
Processed 20 combinations in 0.192000150680542


In [4]:
fr

,combo,frequency,file,checks_count,description
0,"(1, 5, 8)",2175248,data,2908712,Молочная гастрономия-Сопутствующие товары (Foo...
1,"(1, 5, 14)",2158619,data,2908712,Молочная гастрономия-Сопутствующие товары (Foo...
2,"(1, 17, 8)",2120696,data,2908712,"Молочная гастрономия-Овощи - Фрукты-Соки, воды..."
3,"(5, 17, 8)",2111083,data,2908712,Сопутствующие товары (Food)-Овощи - Фрукты-Сок...
4,"(1, 14, 8)",2110397,data,2908712,Молочная гастрономия-Кондитерские изделия (Foo...
5,"(5, 17, 14)",2109720,data,2908712,Сопутствующие товары (Food)-Овощи - Фрукты-Кон...
6,"(1, 5, 17)",2103368,data,2908712,Молочная гастрономия-Сопутствующие товары (Foo...
0,"(1, 5, 17)",395867,data8cat,396544,Молочная гастрономия-Сопутствующие товары (Foo...
1,"(1, 5, 2)",395664,data8cat,396544,Молочная гастрономия-Сопутствующие товары (Foo...
2,"(1, 5, 14)",395306,data8cat,396544,Молочная гастрономия-Сопутствующие товары (Foo...


In [ ]:
fr2=fq.TURF(5,description=True)
fr3=fq.TURF(8,description=True)

In [ ]:
df_save=[['turf3',fr],['turf5',fr2],['turf8',fr3]]
writer = pd.ExcelWriter(path.join(folder,'turf358.xlsx'))
for el in df_save:
    el[1].to_excel(writer,str(el[0]),index =False)

writer.save()

In [ ]:
fr=fq.noTURF(3,100,description=True)
fr2=fq.noTURF(5,100,description=True)
fr3=fq.noTURF(8,100,description=True)
df_save=[['noturf3',fr],['noturf5',fr2],['noturf8',fr3]]
writer = pd.ExcelWriter(path.join(folder,'noturf358.xlsx'))
for el in df_save:
    el[1].to_excel(writer,str(el[0]),index =False)

writer.save()

In [ ]:
df,c=fq.preprocess_df(r'C:\Reports\TURF\Raw\data.csv')
pt= pd.DataFrame(df.pivot_table(values='h2',index=['check_id'], aggfunc=lambda x: len(x.unique())))
pt.reset_index(inplace=True)
rename_check={'check_id':'check_id','h2':'cats_count'}
pt.columns= [rename_check[el] for el in pt.columns.values]
pt=pt[pt['cats_count']>=8]
pt=pd.merge(df, pt, 'inner', on = ['check_id'] )
pt.drop('cats_count',1,inplace=True)
d=fq.category_dict
d= dict(zip(d.values(), d.keys()))
pt['h2']=pt['h2'].apply(lambda x:d[x])
d=  dict(zip(rename_dict.values(), rename_dict.keys()))
pt.columns=[d[el] for el in pt.columns.values]
#pt.to_csv(r'C:\Reports\TURF\Raw\data8cat.csv',sep=';',index=False)

In [ ]:
pt.to_csv(r'C:\Reports\TURF\Raw\data8cat.csv',sep=';',index=False)

In [ ]:
pt=pd.read_csv(r'C:\Reports\TURF\Raw\rat.csv',sep=';')

In [ ]:
def half_divide_method(a, b, f):
    x = (a + b) / 2
    while math.fabs(f(x)) >= e:
        x = (a + b) / 2
        a, b = (a, x) if f(a) * f(x) < 0 else (x, b)
    return (a + b) / 2